In [17]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# 데이터 로드 및 병합
df_price = pd.read_csv("eth_4hour_raw_price.csv")
df_cal = pd.read_csv("eth_4hour_ETH.csv")
df = pd.merge(df_price, df_cal, on='date', how='inner').sort_values('date').reset_index(drop=True)

df=df[df['date']>='2020-01-01']

# 1. 손익비별 라벨 생성 (R=2.0만 먼저)
def create_single_rr_label(df, rr=2.0, horizon=12):
    profit_target = 0.03
    stop_loss = profit_target / rr
    
    indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=horizon)
    future_high = df['ETH_High'].rolling(window=indexer).max()
    future_low = df['ETH_Low'].rolling(window=indexer).min()
    entry_price = df['ETH_Close']
    
    hit_loss_any = (future_low / entry_price - 1) <= -stop_loss
    hit_profit = (future_high / entry_price - 1) >= profit_target
    df[f'Target_R{rr}'] = ((~hit_loss_any) & hit_profit).astype(int)
    
    return df.dropna()

df = create_single_rr_label(df, rr=2.0)

# 2. 피처 선택
def select_features_voting(X, y, feature_names, top_n=30):
    mi_scores = mutual_info_classif(X, y, random_state=42)
    mi_rank = pd.Series(mi_scores, index=feature_names).rank(ascending=False)
    
    rf = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)
    rf.fit(X, y)
    rf_rank = pd.Series(rf.feature_importances_, index=feature_names).rank(ascending=False)
    
    lgbm = LGBMClassifier(n_estimators=200, random_state=42, verbose=-1)
    lgbm.fit(X, y)
    lgbm_rank = pd.Series(lgbm.feature_importances_, index=feature_names).rank(ascending=False)
    
    total_rank = (mi_rank + rf_rank + lgbm_rank) / 3
    return total_rank.sort_values().head(top_n).index.tolist()

# 3. 완전한 WFO + 모델 + 신호 + 수익 계산
def run_complete_wfo_pipeline(df, target_col, train_window=2190, test_window=360, embargo=12, top_n=30, threshold=0.6):
    price_suffixes = ('Open', 'High', 'Low', 'Close', 'Volume', 'Vol')
    all_targets = [c for c in df.columns if c.startswith('Target_')]
    drop_cols.extend(all_targets)
    drop_cols = [c for c in df.columns if c.endswith(price_suffixes)]
    drop_cols.append('date')
    drop_cols.append(target_col)
    feature_candidates = [c for c in df.columns if c not in drop_cols]
    
    results = []
    total_len = len(df)
    
    for i in range(train_window + embargo, total_len, test_window):
        train_end = i - embargo
        test_end = min(i + test_window, total_len)
        
        train_data = df.iloc[i-train_window:train_end]
        test_data = df.iloc[i:test_end]
        
        if len(test_data) < 10:  # 최소 테스트 데이터 확보
            break
            
        X_train_full = train_data[feature_candidates].fillna(0)
        y_train = train_data[target_col]
        
        selected_features = select_features_voting(X_train_full, y_train, feature_candidates, top_n)
        
        scaler = StandardScaler()
        X_train = scaler.fit_transform(train_data[selected_features].fillna(0))
        X_test = scaler.transform(test_data[selected_features].fillna(0))
        
        model = LGBMClassifier(n_estimators=500, learning_rate=0.03, random_state=42, class_weight='balanced', verbose=-1)
        model.fit(X_train, y_train)
        prob_test = model.predict_proba(X_test)[:, 1]
        
        signal = (prob_test >= threshold).astype(int)
        
        # 수익 계산 (R=2.0 기준: +3%, -1.5%)
        FEE = 0.0008
        strategy_return = np.zeros(len(test_data))
        profit_target = 0.03
        stop_loss = profit_target / 2.0
        
        for j, actual in enumerate(test_data[target_col]):
            if signal[j] == 1:
                if actual == 1:
                    strategy_return[j] = profit_target - FEE
                else:
                    strategy_return[j] = -stop_loss - FEE
        
        chunk_res = pd.DataFrame({
            'date': test_data['date'].values,
            'actual': test_data[target_col].values,
            'prob': prob_test,
            'signal': signal,
            'strategy_return': strategy_return
        })
        results.append(chunk_res)
    
    return pd.concat(results).reset_index(drop=True)

# 4. 실행
target_col = 'Target_R2.0'
final_df = run_complete_wfo_pipeline(df, target_col)




=== Target Distribution ===
Target_Direction (단순방향): 52.4%
Target_Easy (1.5%/-1%): 15.3%
Target_Trend (추세+2%/-1.5%): 12.3%


In [38]:
import pandas as pd
import numpy as np
# 데이터 로드 및 병합
df_price = pd.read_csv("eth_4hour_raw_price.csv")
df_cal = pd.read_csv("eth_4hour_ETH.csv")
df = pd.merge(df_price, df_cal, on='date', how='inner').sort_values('date').reset_index(drop=True)

df=df[df['date']>='2020-01-01']
def create_alternative_targets_next_open(df, horizon=12):
    # 원본 데이터 보존
    df = df.copy()
    
    # 1. 진입가: 다음 봉의 시가 (t+1 Open)
    # 모델은 t시점(오늘 종가)까지의 데이터를 보고, t+1시점 시가에 진입한다고 가정
    entry_price = df['ETH_Open'].shift(-1)
    
    # 2. 고가/저가 윈도우 설정 (중요!)
    # 진입이 t+1이므로, 수익/손실 감지 윈도우도 t+1부터 시작해야 함
    # 따라서 High/Low 데이터를 미리 한 칸씩 당겨(shift -1) 놓고 롤링을 돌려야 함
    next_high = df['ETH_High'].shift(-1)
    next_low = df['ETH_Low'].shift(-1)
    
    indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=horizon)
    
    # 윈도우 내 최고가/최저가 (t+1 ~ t+horizon 구간)
    future_high = next_high.rolling(window=indexer).max()
    future_low = next_low.rolling(window=indexer).min()
    
    # 3. 청산가: N봉 뒤의 종가
    # shift(-horizon)은 t 시점 기준으로 t+horizon 번째 종가를 가져옴
    future_close = df['ETH_Close'].shift(-horizon)
    
    # --- 타겟 생성 로직 (이전과 동일, 기준 가격만 entry_price로 변경됨) ---
    
    # 타겟1: 단순 방향 (진입가 대비 N봉 후 종가가 높으면 1)
    df['Target_Direction'] = (future_close > entry_price).astype(int)
    
    # 타겟2: 부드러운 수익 (익절 1.5% 도달 & 손절 -1% 미도달)
    hit_profit_easy = (future_high / entry_price - 1) >= 0.015
    hit_loss_tight = (future_low / entry_price - 1) <= -0.01
    df['Target_Easy'] = ((~hit_loss_tight) & hit_profit_easy).astype(int)
    
    # 타겟3: 추세 필터 (EMA 필터는 '판단 시점인 t' 기준이므로 그대로 둠)
    # t시점에 이미 추세가 좋아야 진입한다는 논리
    df['EMA20'] = df['ETH_Close'].ewm(span=20).mean()
    trend_up = df['ETH_Close'] > df['EMA20']
    
    hit_profit_mid = (future_high / entry_price - 1) >= 0.02
    hit_loss_mid = (future_low / entry_price - 1) <= -0.015
    df['Target_Trend'] = (trend_up & (~hit_loss_mid) & hit_profit_mid).astype(int)
    
    return df.dropna()
df = create_alternative_targets_next_open(df, horizon=12)

In [31]:
targets_to_test = ['Target_Direction', 'Target_Easy', 'Target_Trend']
rr_map = {'Target_Direction': 1.0, 'Target_Easy': 1.5, 'Target_Trend': 1.33}
pt_map = {'Target_Direction': 0.015, 'Target_Easy': 0.015, 'Target_Trend': 0.02}

all_results = {}

for target_col in targets_to_test:
    print(f"\n>>> Testing {target_col}...")
    result_df = run_complete_wfo_pipeline(df, target_col, threshold=0.5)
    result_df = result_df.merge(df[['date', 'ETH_Close']], on='date', how='left')
    
    rr = rr_map[target_col]
    pt = pt_map[target_col]
    sl = pt / rr
    FEE = 0.0008
    
    result_df['strategy_return'] = 0.0
    mask_win = (result_df['signal'] == 1) & (result_df['actual'] == 1)
    mask_lose = (result_df['signal'] == 1) & (result_df['actual'] == 0)
    result_df.loc[mask_win, 'strategy_return'] = pt - FEE
    result_df.loc[mask_lose, 'strategy_return'] = -sl - FEE
    
    result_df['cum_return'] = (1 + result_df['strategy_return']).cumprod()
    
    trades = result_df[result_df['signal'] == 1]
    win_rate = (trades['strategy_return'] > 0).mean() if len(trades) > 0 else 0
    final_ret = result_df['cum_return'].iloc[-1]
    max_dd = (result_df['cum_return'] / result_df['cum_return'].cummax() - 1).min()
    
    all_results[target_col] = {
        'trades': len(trades),
        'win_rate': win_rate,
        'final_return': final_ret,
        'max_dd': max_dd
    }

print("\n" + "="*50)
print("=== All Targets Comparison ===")
print("="*50)
for t, r in all_results.items():
    print(f"\n{t}:")
    print(f"  Trades: {r['trades']}, WinRate: {r['win_rate']:.1%}")
    print(f"  Return: {r['final_return']:.4f}, MaxDD: {r['max_dd']:.4f}")




>>> Testing Target_Direction...

>>> Testing Target_Easy...

>>> Testing Target_Trend...

=== All Targets Comparison ===

Target_Direction:
  Trades: 5877, WinRate: 64.1%
  Return: 280178742.6203, MaxDD: -0.8995

Target_Easy:
  Trades: 2442, WinRate: 59.3%
  Return: 15114.1886, MaxDD: -0.2781

Target_Trend:
  Trades: 1408, WinRate: 83.9%
  Return: 151539627.7110, MaxDD: -0.2694


In [36]:
# Target_Trend 모델이 가장 중요하게 생각하는 변수 TOP 10 출력
# (run_complete_wfo_pipeline 함수 내부 혹은 루프 안에서 실행)

# 예시: 마지막 fold의 모델을 꺼내올 수 없다면, 전체 데이터로 한 번만 다시 학습해서 확인
import xgboost as xgb
X = df[feature_cols]
y = df['Target_Trend']

model = xgb.XGBClassifier(n_estimators=100, max_depth=5)
model.fit(X, y)

# 중요도 출력
feature_imp = pd.Series(model.feature_importances_, index=feature_cols).sort_values(ascending=False)
print("\n>>> [충격] 모델이 가장 사랑한 변수 TOP 10:")
print(feature_imp.head(10))

NameError: name 'feature_cols' is not defined

In [40]:
import pandas as pd
import numpy as np

# (데이터 로드 및 함수 정의는 이전과 동일)
df = create_alternative_targets_next_open(df, horizon=12)
targets_to_test = ['Target_Direction', 'Target_Easy', 'Target_Trend']
rr_map = {'Target_Direction': 1.0, 'Target_Easy': 1.5, 'Target_Trend': 1.33}
pt_map = {'Target_Direction': 0.015, 'Target_Easy': 0.015, 'Target_Trend': 0.02}

all_results = {}

# 3. 완전한 WFO + 모델 + 신호 + 수익 계산
def run_complete_wfo_pipeline(df, target_col, train_window=2190, test_window=360, embargo=12, top_n=30, threshold=0.6):
    price_suffixes = ('Open', 'High', 'Low', 'Close', 'Volume', 'Vol')
    all_targets = [c for c in df.columns if c.startswith('Target_')]
    
    drop_cols = [c for c in df.columns if c.endswith(price_suffixes)]
    drop_cols.extend(all_targets)
    drop_cols.append('date')
    drop_cols.append(target_col)
    feature_candidates = [c for c in df.columns if c not in drop_cols]
    
    results = []
    total_len = len(df)
    
    for i in range(train_window + embargo, total_len, test_window):
        train_end = i - embargo
        test_end = min(i + test_window, total_len)
        
        train_data = df.iloc[i-train_window:train_end]
        test_data = df.iloc[i:test_end]
        
        if len(test_data) < 10:  # 최소 테스트 데이터 확보
            break
            
        X_train_full = train_data[feature_candidates].fillna(0)
        y_train = train_data[target_col]
        
        selected_features = select_features_voting(X_train_full, y_train, feature_candidates, top_n)
        
        scaler = StandardScaler()
        X_train = scaler.fit_transform(train_data[selected_features].fillna(0))
        X_test = scaler.transform(test_data[selected_features].fillna(0))
        
        model = LGBMClassifier(n_estimators=500, learning_rate=0.03, random_state=42, class_weight='balanced', verbose=-1)
        model.fit(X_train, y_train)
        prob_test = model.predict_proba(X_test)[:, 1]
        
        signal = (prob_test >= threshold).astype(int)
        
        # 수익 계산 (R=2.0 기준: +3%, -1.5%)
        FEE = 0.0008
        strategy_return = np.zeros(len(test_data))
        profit_target = 0.03
        stop_loss = profit_target / 2.0
        
        for j, actual in enumerate(test_data[target_col]):
            if signal[j] == 1:
                if actual == 1:
                    strategy_return[j] = profit_target - FEE
                else:
                    strategy_return[j] = -stop_loss - FEE
        
        chunk_res = pd.DataFrame({
            'date': test_data['date'].values,
            'actual': test_data[target_col].values,
            'prob': prob_test,
            'signal': signal,
            'strategy_return': strategy_return
        })
        results.append(chunk_res)
    
    return pd.concat(results).reset_index(drop=True)

for target_col in targets_to_test:
    print(f"\n>>> Testing {target_col}...")
    result_df = run_complete_wfo_pipeline(df, target_col, threshold=0.5)
    result_df = result_df.merge(df[['date', 'ETH_Close']], on='date', how='left')
    
    rr = rr_map[target_col]
    pt = pt_map[target_col]
    sl = pt / rr
    FEE = 0.0008
    
    result_df['strategy_return'] = 0.0
    mask_win = (result_df['signal'] == 1) & (result_df['actual'] == 1)
    mask_lose = (result_df['signal'] == 1) & (result_df['actual'] == 0)
    result_df.loc[mask_win, 'strategy_return'] = pt - FEE
    result_df.loc[mask_lose, 'strategy_return'] = -sl - FEE
    
    # [수정 1] cumprod() 대신 cumsum()을 사용하여 '단리' 누적 수익률 계산
    # 이렇게 하면 수익률을 단순히 더해가므로 현실적인 수치가 나옵니다.
    result_df['cum_return'] = result_df['strategy_return'].cumsum()
    
    trades = result_df[result_df['signal'] == 1]
    win_rate = (trades['strategy_return'] > 0).mean() if len(trades) > 0 else 0
    
    # 최종 Return은 이제 수익률의 총합이 됩니다.
    final_ret = result_df['cum_return'].iloc[-1]
    
    # [수정 2] MaxDD 계산 방식도 단리에 맞게 변경
    # (누적 수익률의 최고점 - 현재 누적 수익률) 중 가장 큰 값
    drawdown = result_df['cum_return'].cummax() - result_df['cum_return']
    max_dd = drawdown.max()
    
    all_results[target_col] = {
        'trades': len(trades),
        'win_rate': win_rate,
        # final_return의 단위는 이제 %가 아닌 단순 합계입니다. 
        # 예: 25.4 라는 값이 나오면 2540% 총수익을 의미합니다.
        'final_return': final_ret,
        'max_dd': max_dd
    }

print("\n" + "="*50)
print("=== All Targets Comparison (Corrected) ===")
print("="*50)
for t, r in all_results.items():
    # Return과 MaxDD에 100을 곱해 퍼센트로 표시하면 더 보기 좋습니다.
    print(f"\n{t}:")
    print(f"  Trades: {r['trades']}, WinRate: {r['win_rate']:.1%}")
    print(f"  Total Return: {r['final_return'] * 100:.2f}%, MaxDD: {r['max_dd'] * 100:.2f}%")



>>> Testing Target_Direction...

>>> Testing Target_Easy...

>>> Testing Target_Trend...

=== All Targets Comparison (Corrected) ===

Target_Direction:
  Trades: 5682, WinRate: 50.7%
  Total Return: -328.56%, MaxDD: 493.68%

Target_Easy:
  Trades: 1860, WinRate: 20.8%
  Total Return: -1041.30%, MaxDD: 1041.64%

Target_Trend:
  Trades: 934, WinRate: 23.7%
  Total Return: -704.90%, MaxDD: 706.82%


In [33]:
# 1. 시간별 수익 분포 확인 (언제 벌었나?)
trend_result = run_complete_wfo_pipeline(df, 'Target_Trend', threshold=0.5)
trend_result = trend_result.merge(df[['date', 'ETH_Close']], on='date', how='left')

# 수익 재계산
pt, sl, FEE = 0.02, 0.015, 0.0008
trend_result['strategy_return'] = 0.0
mask_win = (trend_result['signal'] == 1) & (trend_result['actual'] == 1)
mask_lose = (trend_result['signal'] == 1) & (trend_result['actual'] == 0)
trend_result.loc[mask_win, 'strategy_return'] = pt - FEE
trend_result.loc[mask_lose, 'strategy_return'] = -sl - FEE
trend_result['cum_return'] = (1 + trend_result['strategy_return']).cumprod()

# 2. 연도별 성과 확인
trend_result['year'] = pd.to_datetime(trend_result['date']).dt.year
yearly = trend_result.groupby('year').agg({
    'signal': 'sum',
    'strategy_return': lambda x: (1 + x).prod() - 1
}).rename(columns={'signal': 'trades', 'strategy_return': 'return'})
print("=== Yearly Performance (Target_Trend) ===")
print(yearly)

# 3. 최근 1년만 따로 확인
recent = trend_result[pd.to_datetime(trend_result['date']) >= '2024-01-01']
recent_trades = recent[recent['signal'] == 1]
print(f"\n=== 2024~ Only ===")
print(f"Trades: {len(recent_trades)}")
print(f"Win Rate: {(recent_trades['strategy_return'] > 0).mean():.1%}")
print(f"Return: {(1 + recent['strategy_return']).prod():.4f}")

=== Yearly Performance (Target_Trend) ===
      trades     return
year                   
2021     298  39.873952
2022     208  23.215036
2023     335  94.005599
2024     287  46.027685
2025     280  33.566661

=== 2024~ Only ===
Trades: 567
Win Rate: 82.9%
Return: 1625.5901


In [34]:
# 1. 시간별 수익 분포 확인 (언제 벌었나?)
trend_result = run_complete_wfo_pipeline(df, 'Target_Trend', threshold=0.5)
trend_result = trend_result.merge(df[['date', 'ETH_Close']], on='date', how='left')

# 수익 재계산
pt, sl, FEE = 0.02, 0.015, 0.0008
trend_result['strategy_return'] = 0.0
mask_win = (trend_result['signal'] == 1) & (trend_result['actual'] == 1)
mask_lose = (trend_result['signal'] == 1) & (trend_result['actual'] == 0)
trend_result.loc[mask_win, 'strategy_return'] = pt - FEE
trend_result.loc[mask_lose, 'strategy_return'] = -sl - FEE
trend_result['cum_return'] = (1 + trend_result['strategy_return']).cumprod()

# 2. 연도별 성과 확인
trend_result['year'] = pd.to_datetime(trend_result['date']).dt.year
yearly = trend_result.groupby('year').agg({
    'signal': 'sum',
    'strategy_return': lambda x: (1 + x).prod() - 1
}).rename(columns={'signal': 'trades', 'strategy_return': 'return'})
print("=== Yearly Performance (Target_Trend) ===")
print(yearly)

# 3. 최근 1년만 따로 확인
recent = trend_result[pd.to_datetime(trend_result['date']) >= '2024-01-01']
recent_trades = recent[recent['signal'] == 1]
print(f"\n=== 2024~ Only ===")
print(f"Trades: {len(recent_trades)}")
print(f"Win Rate: {(recent_trades['strategy_return'] > 0).mean():.1%}")
print(f"Return: {(1 + recent['strategy_return']).prod():.4f}")


=== Yearly Performance (Target_Trend) ===
      trades     return
year                   
2021     298  39.873952
2022     208  23.215036
2023     335  94.005599
2024     287  46.027685
2025     280  33.566661

=== 2024~ Only ===
Trades: 567
Win Rate: 82.9%
Return: 1625.5901


In [20]:
# 1. 월별 성과 확인 (연속 손실 구간 있는지)
trend_result['month'] = pd.to_datetime(trend_result['date']).dt.to_period('M')
monthly = trend_result.groupby('month').agg({
    'signal': 'sum',
    'strategy_return': lambda x: (1 + x).prod() - 1
})
monthly.columns = ['trades', 'return']
losing_months = (monthly['return'] < 0).sum()
print(f"Total Months: {len(monthly)}")
print(f"Losing Months: {losing_months}")
print(f"Win Month Rate: {1 - losing_months/len(monthly):.1%}")

# 2. 최대 연속 손실 트레이드 수
trades_only = trend_result[trend_result['signal'] == 1].copy()
trades_only['is_loss'] = trades_only['strategy_return'] < 0
max_consecutive_loss = (trades_only['is_loss'].groupby(
    (~trades_only['is_loss']).cumsum()
).cumsum()).max()
print(f"Max Consecutive Losses: {max_consecutive_loss}")

# 3. 평균 트레이드당 수익
avg_return = trades_only['strategy_return'].mean()
print(f"Avg Return per Trade: {avg_return:.4f} ({avg_return*100:.2f}%)")


Total Months: 60
Losing Months: 7
Win Month Rate: 88.3%
Max Consecutive Losses: 17
Avg Return per Trade: 0.0105 (1.05%)


In [21]:
# 모델 없이 "타겟=1이면 무조건 진입"했을 때 결과
trend_result['signal_oracle'] = trend_result['actual']  # 타겟 그대로 시그널로

pt, sl, FEE = 0.02, 0.015, 0.0008
trend_result['return_oracle'] = 0.0
trend_result.loc[trend_result['signal_oracle'] == 1, 'return_oracle'] = pt - FEE

oracle_cum = (1 + trend_result['return_oracle']).prod()
oracle_trades = trend_result['signal_oracle'].sum()

print("=== Oracle (타겟=1이면 무조건 진입) ===")
print(f"Trades: {oracle_trades}")
print(f"Win Rate: 100% (정의상)")
print(f"Final Return: {oracle_cum:.4f}")

print("\n=== Model (실제 모델 예측) ===")
model_trades = trend_result['signal'].sum()
model_wins = ((trend_result['signal']==1) & (trend_result['actual']==1)).sum()
print(f"Trades: {model_trades}")
print(f"Win Rate: {model_wins/model_trades:.1%}")


=== Oracle (타겟=1이면 무조건 진입) ===
Trades: 1276
Win Rate: 100% (정의상)
Final Return: 34595015447.8841

=== Model (실제 모델 예측) ===
Trades: 1040
Win Rate: 75.2%


In [22]:
# 보수적 추정: 승률 75% → 실전 65%, 손익도 80%만 실현
real_win_rate = 0.65
real_profit = 0.02 * 0.8
real_loss = 0.015 * 1.2

# 1년 트레이드 약 200회 가정
trades_year = 200
wins_year = trades_year * real_win_rate
losses_year = trades_year * (1 - real_win_rate)

yearly_return = (1 + real_profit)**wins_year * (1 - real_loss)**losses_year
print(f"현실적 연간 기대수익: {yearly_return:.2f}배 ({(yearly_return-1)*100:.0f}%)")


현실적 연간 기대수익: 2.21배 (121%)
